In [ ]:
%matplotlib inline
import cv2
import keras
import numpy as np
import os
from matplotlib import pyplot
from matplotlib import image
from scipy.misc import toimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=60,
        #rescale=1./255,
        shear_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

file_path = '../sampletrain/validation/BET/'

for file in os.listdir(file_path):
    if file.endswith('.jpg'):
        this_file = file_path + file
        img = load_img(this_file)  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

        # the .flow() command below generates batches of randomly transformed images
        # and saves the results to the `preview/` directory
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir='../sampletrain/train/BET', save_prefix='fish', save_format='jpg'):
            i += 1
            if i > 3:
                break  # otherwise the generator would loop indefinitely

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '../sampletrain/train'
validation_data_dir = '../sampletrain/validation'
nb_train_samples = 2000
nb_validation_samples = 800
nb_epoch = 50


model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

model.save_weights('first_try.h5')

Found 42 images belonging to 2 classes.
Found 42 images belonging to 2 classes.
Epoch 1/50
2006/2000 [==============================] - 424s - loss: 0.2408 - acc: 0.9192 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 2/50
 136/2000 [=>............................] - ETA: 326s - loss: 0.0125 - acc: 1.0000

/local/java/python-pip-packages/lib/python2.7/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


KeyboardInterrupt: 